# Resource Validation

> Validate resource availability before job execution and determine appropriate actions

In [ ]:
#| default_exp core.validation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass

from cjm_fasthtml_resources.core.manager import (
    ResourceStatus,
    ResourceConflict,
    WorkerState
)

# Optional: Import error handling library if available
try:
    from cjm_error_handling.core.base import ErrorContext, ErrorSeverity
    from cjm_error_handling.core.errors import ValidationError, ResourceError
    _has_error_handling = True
except ImportError:
    _has_error_handling = False

## Validation Actions

The validation system returns different actions based on resource availability:

In [ ]:
#| export
class ValidationAction(Enum):
    """Actions that can be taken based on validation results."""
    PROCEED = "proceed"  # Resource available, proceed with job
    RELOAD_PLUGIN = "reload_plugin"  # Reload plugin (for switching plugins or resource configs)
    WAIT_FOR_JOB = "wait_for_job"  # Wait for current job to complete (same worker type)
    WAIT_FOR_OTHER_WORKER = "wait_for_other_worker"  # Wait for different worker type to finish
    USER_INTERVENTION = "user_intervention"  # External process conflict, need user action
    ABORT = "abort"  # Cannot proceed

## Validation Result

The result object contains all information needed to make decisions about job execution:

In [ ]:
#| export
@dataclass
class ValidationResult:
    """Result of resource validation."""
    action: ValidationAction
    can_proceed: bool
    message: str
    conflict: Optional[ResourceConflict] = None
    current_worker: Optional[WorkerState] = None
    plugin_name_to_reload: Optional[str] = None  # Plugin name to reload
    new_config: Optional[Dict[str, Any]] = None  # Config for reload

In [ ]:
# Example: Creating a validation result
result = ValidationResult(
    action=ValidationAction.PROCEED,
    can_proceed=True,
    message="GPU available. Proceeding with job."
)

print(f"Action: {result.action.value}")
print(f"Can proceed: {result.can_proceed}")
print(f"Message: {result.message}")

Action: proceed
Can proceed: True
Message: GPU available. Proceeding with job.


## Resource Validation Function

This is the main validation function that coordinates resource checks and returns appropriate actions.

The validation logic handles several scenarios:

1. **CPU-based plugins**: Check system memory availability
2. **API-based plugins**: Skip resource validation
3. **GPU-based plugins**: Check GPU availability and handle conflicts
4. **Plugin switching**: Detect when plugins need to be reloaded
5. **Resource conflicts**: Identify conflicts with app workers or external processes

In [ ]:
#| export
def validate_resources_for_job(
    resource_manager, # ResourceManager instance
    plugin_registry, # Plugin registry protocol (has get_plugin, load_plugin_config methods)
    get_plugin_resource_requirements, # Function: (plugin_id, config) -> Dict with requirements
    compare_plugin_resources, # Function: (config1, config2) -> bool (same resource?)
    get_plugin_resource_identifier, # Function: (config) -> str (resource ID)
    plugin_id:str, # Unique plugin ID
    plugin_config:Optional[Dict[str, Any]]=None, # Plugin configuration (will load if not provided)
    worker_pid:Optional[int]=None, # PID of the worker that will run the job (if known)
    worker_type:str="transcription", # Type of worker (e.g., "transcription", "llm", "ollama")
    verbose:bool=False # Whether to print verbose logging
) -> ValidationResult: # ValidationResult with action to take
    """Validate if resources are available to run a job with the specified plugin. This function is dependency-injected with helper functions to avoid tight coupling with specific plugin registry implementations."""
    if verbose:
        print(f"\n=== Resource Validation Started ===")
        print(f"Plugin ID: {plugin_id}")
        print(f"Worker PID: {worker_pid}")
        print(f"Worker Type: {worker_type}")

    # Get plugin resource requirements
    requirements = get_plugin_resource_requirements(plugin_id, plugin_config)
    if verbose:
        print(f"Plugin requirements: {requirements}")

    # Check system memory if using CPU
    if requirements['is_local'] and requirements['device'].lower() == 'cpu':
        if verbose:
            print("→ CPU-based plugin, checking system memory...")

        # TODO: Add comprehensive system memory validation
        # For now, just log current memory status
        try:
            import psutil
            mem = psutil.virtual_memory()
            if verbose:
                print(f"   System memory: {mem.percent}% used ({mem.used / (1024**3):.1f}GB / {mem.total / (1024**3):.1f}GB)")

            # Placeholder warning (can be enhanced with thresholds)
            if mem.percent > 85:
                if verbose:
                    print(f"   ⚠ System memory high: {mem.percent}% used")
        except ImportError:
            if verbose:
                print("   ⚠ psutil not available, skipping memory check")

        # For now, always proceed for CPU plugins
        if verbose:
            print("✓ System memory check passed (basic)")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="CPU-based plugin. System memory check passed."
        )

    # If not a local plugin or doesn't use GPU, no validation needed
    if not requirements['is_local'] or not requirements['uses_gpu']:
        if verbose:
            print("✓ No GPU resources required")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="No GPU resources required. Proceeding with job."
        )

    # Load plugin config if not provided
    if plugin_config is None:
        plugin_config = plugin_registry.load_plugin_config(plugin_id)

    # Get plugin metadata
    plugin_meta = plugin_registry.get_plugin(plugin_id)
    if not plugin_meta:
        return ValidationResult(
            action=ValidationAction.ABORT,
            can_proceed=False,
            message=f"Plugin {plugin_id} not found."
        )

    plugin_name = plugin_meta.name
    if verbose:
        print(f"Plugin name: {plugin_name}")

    # Check GPU availability
    gpu_conflict = resource_manager.check_gpu_availability()
    if verbose:
        print(f"GPU status: {gpu_conflict.status.value}")
        print(f"App PIDs using GPU: {gpu_conflict.app_pids}")
        print(f"External PIDs using GPU: {gpu_conflict.external_pids}")

    # Case 1: GPU is available (no processes using it)
    if gpu_conflict.status == ResourceStatus.AVAILABLE:
        if verbose:
            print("GPU Status: AVAILABLE")
        # Check if worker already has a plugin loaded
        if worker_pid:
            worker = resource_manager.get_worker_by_pid(worker_pid)
            if worker and worker.plugin_name:
                if verbose:
                    print(f"Worker has plugin loaded: {worker.plugin_name}")
                if worker.plugin_name == plugin_name:
                    # Same plugin - check if same plugin resource
                    if worker.config and compare_plugin_resources(worker.config, plugin_config):
                        # Same plugin, same resource - proceed
                        if verbose:
                            print(f"✓ Same plugin and resource already loaded")
                            print("Action: PROCEED")
                            print("=== Resource Validation Complete ===\n")
                        return ValidationResult(
                            action=ValidationAction.PROCEED,
                            can_proceed=True,
                            message="Plugin and resource already loaded. Proceeding with job.",
                            current_worker=worker
                        )
                    else:
                        # Same plugin, different resource - reload with new config
                        old_resource = get_plugin_resource_identifier(worker.config) if worker.config else "unknown"
                        new_resource = get_plugin_resource_identifier(plugin_config)
                        if verbose:
                            print(f"→ Resource change detected: {old_resource} → {new_resource}")
                            print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                            print("=== Resource Validation Complete ===\n")
                        return ValidationResult(
                            action=ValidationAction.RELOAD_PLUGIN,
                            can_proceed=True,
                            message=f"Reloading {plugin_name} with new resource configuration.",
                            current_worker=worker,
                            plugin_name_to_reload=plugin_name,
                            new_config=plugin_config
                        )
                else:
                    # Different plugin - reload to switch plugins
                    if verbose:
                        print(f"→ Plugin switch: {worker.plugin_name} → {plugin_name}")
                        print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.RELOAD_PLUGIN,
                        can_proceed=True,
                        message=f"Switching from {worker.plugin_name} to {plugin_name}.",
                        current_worker=worker,
                        plugin_name_to_reload=plugin_name,
                        new_config=plugin_config
                    )

        # GPU available, no plugin loaded - proceed
        if verbose:
            print("✓ No plugin loaded, GPU available")
            print("Action: PROCEED")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="GPU available. Proceeding with job."
        )

    # Case 2: GPU busy with application process
    elif gpu_conflict.status == ResourceStatus.APP_BUSY:
        if verbose:
            print("GPU Status: APP_BUSY")
        # Find which worker is using GPU
        app_pids = gpu_conflict.app_pids
        if not app_pids:
            # Shouldn't happen, but handle gracefully
            if verbose:
                print("⚠ GPU reported as busy but no app PIDs found")
                print("Action: PROCEED (fallback)")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.PROCEED,
                can_proceed=True,
                message="GPU reported as busy but no app PIDs found. Proceeding."
            )

        # Get worker using GPU
        gpu_worker_pid = app_pids[0]  # Assuming single GPU, single worker
        if verbose:
            print(f"GPU in use by app worker PID: {gpu_worker_pid}")
        gpu_worker = resource_manager.get_worker_by_pid(gpu_worker_pid)

        if not gpu_worker:
            # Unknown worker, proceed cautiously
            if verbose:
                print(f"⚠ Unknown worker PID {gpu_worker_pid}")
                print("Action: PROCEED (cautiously)")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.PROCEED,
                can_proceed=True,
                message=f"GPU in use by app PID {gpu_worker_pid}. Proceeding anyway.",
                conflict=gpu_conflict
            )

        if verbose:
            print(f"Worker status: {gpu_worker.status}")
            print(f"Worker plugin: {gpu_worker.plugin_name}")

        # Check if it's the same worker we're going to use
        if worker_pid and gpu_worker_pid == worker_pid:
            if verbose:
                print("→ Same worker will be reused")
            # Same worker - check plugin and resource
            if gpu_worker.plugin_name == plugin_name:
                # Same plugin - check resource
                if gpu_worker.config and compare_plugin_resources(gpu_worker.config, plugin_config):
                    # Same plugin, same resource
                    if verbose:
                        print("✓ Same plugin and resource already loaded")
                        print("Action: PROCEED")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.PROCEED,
                        can_proceed=True,
                        message="Plugin and resource already loaded. Proceeding with job.",
                        current_worker=gpu_worker
                    )
                else:
                    # Same plugin, different resource - reload with new config
                    old_resource = get_plugin_resource_identifier(gpu_worker.config) if gpu_worker.config else "unknown"
                    new_resource = get_plugin_resource_identifier(plugin_config)
                    if verbose:
                        print(f"→ Resource change detected: {old_resource} → {new_resource}")
                        print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.RELOAD_PLUGIN,
                        can_proceed=True,
                        message=f"Reloading {plugin_name} with new resource configuration.",
                        current_worker=gpu_worker,
                        plugin_name_to_reload=plugin_name,
                        new_config=plugin_config
                    )
            else:
                # Different plugin - reload to switch
                if verbose:
                    print(f"→ Plugin switch: {gpu_worker.plugin_name} → {plugin_name}")
                    print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.RELOAD_PLUGIN,
                    can_proceed=True,
                    message=f"Switching from {gpu_worker.plugin_name} to {plugin_name}.",
                    current_worker=gpu_worker,
                    plugin_name_to_reload=plugin_name,
                    new_config=plugin_config
                )

        # Different worker using GPU
        if verbose:
            print(f"→ Different worker is using GPU (type: {gpu_worker.worker_type})")

        # Check if it's a different worker TYPE
        if gpu_worker.worker_type != worker_type:
            if verbose:
                print(f"→ Cross-worker-type conflict: {gpu_worker.worker_type} ↔ {worker_type}")

            if gpu_worker.status == "running":
                # Different worker type is actively running
                if verbose:
                    print(f"✗ {gpu_worker.worker_type.capitalize()} worker is actively running")
                    print("Action: WAIT_FOR_OTHER_WORKER")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.WAIT_FOR_OTHER_WORKER,
                    can_proceed=False,
                    message=f"GPU in use by running {gpu_worker.worker_type} job (PID {gpu_worker_pid}). Wait for completion or cancel that job.",
                    current_worker=gpu_worker,
                    conflict=gpu_conflict
                )
            else:
                # Different worker type is idle - could stop it
                if verbose:
                    print(f"⚠ {gpu_worker.worker_type.capitalize()} worker is idle but using GPU")
                    print("Action: WAIT_FOR_OTHER_WORKER")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.WAIT_FOR_OTHER_WORKER,
                    can_proceed=False,
                    message=f"GPU in use by idle {gpu_worker.worker_type} worker (PID {gpu_worker_pid}). Stop that worker to free GPU.",
                    current_worker=gpu_worker,
                    conflict=gpu_conflict
                )

        # Same worker type but different instance
        if gpu_worker.status == "running":
            # Worker is actively running a job
            if verbose:
                print("✗ Worker is actively running a job")
                print("Action: WAIT_FOR_JOB")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.WAIT_FOR_JOB,
                can_proceed=False,
                message=f"GPU in use by running job (PID {gpu_worker_pid}). Wait for completion or cancel job.",
                current_worker=gpu_worker,
                conflict=gpu_conflict
            )
        else:
            # Worker is idle but has resource loaded - can reload to switch
            if verbose:
                print(f"Worker is idle, switching to {plugin_name}")
                print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.RELOAD_PLUGIN,
                can_proceed=True,
                message=f"Switching from idle plugin on worker {gpu_worker_pid} to {plugin_name}.",
                current_worker=gpu_worker,
                plugin_name_to_reload=plugin_name,
                new_config=plugin_config,
                conflict=gpu_conflict
            )

    # Case 3: GPU busy with external process
    elif gpu_conflict.status == ResourceStatus.EXTERNAL_BUSY:
        if verbose:
            print("GPU Status: EXTERNAL_BUSY")
        external_processes = gpu_conflict.external_processes
        process_names = [p.get('name', 'unknown') for p in external_processes[:3]]
        if verbose:
            print(f"✗ External processes using GPU: {', '.join(process_names)}")
            for proc in external_processes[:3]:
                print(f"  - PID {proc.get('pid')}: {proc.get('name')} ({proc.get('gpu_memory_mb', 0)} MB)")
            print("Action: USER_INTERVENTION")
            print("=== Resource Validation Complete ===\n")

        return ValidationResult(
            action=ValidationAction.USER_INTERVENTION,
            can_proceed=False,
            message=f"GPU in use by external process(es): {', '.join(process_names)}. User intervention required.",
            conflict=gpu_conflict
        )

    # Fallback
    if verbose:
        print("⚠ Unknown GPU status")
        print("Action: ABORT")
        print("=== Resource Validation Complete ===\n")
    return ValidationResult(
        action=ValidationAction.ABORT,
        can_proceed=False,
        message="Unknown GPU status. Cannot proceed.",
        conflict=gpu_conflict
    )

The validation function uses dependency injection to avoid tight coupling. You provide helper functions that know how to work with your specific plugin registry implementation.

## Error Handling Integration

When the `cjm-error-handling` library is installed, you can convert `ValidationResult` objects into structured exceptions. This is useful when you want to raise errors instead of returning result objects.

In [ ]:
#| export
def validation_result_to_error(
    result: ValidationResult,  # Validation result to convert
    plugin_id: Optional[str] = None,  # Plugin ID for error context
    job_id: Optional[str] = None,  # Job ID for error context
    worker_pid: Optional[int] = None,  # Worker PID for error context
    **extra_context  # Additional context fields
):
    """
    Convert a ValidationResult into a structured error.
    
    This helper allows you to convert validation results into exceptions
    when you prefer raising errors over returning result objects.
    
    Requires: cjm-error-handling library
    
    Example:
        ```python
        result = validate_resources_for_job(...)
        if not result.can_proceed:
            error = validation_result_to_error(result, plugin_id="whisper_large")
            raise error
        ```
    
    Returns appropriate error type based on ValidationAction:
    - ABORT, USER_INTERVENTION -> ValidationError (not retryable)
    - WAIT_FOR_JOB, WAIT_FOR_OTHER_WORKER -> ResourceError (retryable)
    - RELOAD_PLUGIN -> Returns None (this is a success case)
    - PROCEED -> Returns None (this is a success case)
    """
    if not _has_error_handling:
        raise ImportError(
            "cjm-error-handling library not installed. "
            "Install it with: pip install cjm-error-handling"
        )
    
    # Build error context
    ctx_kwargs = {}
    if plugin_id:
        ctx_kwargs['plugin_id'] = plugin_id
    if job_id:
        ctx_kwargs['job_id'] = job_id
    if worker_pid:
        ctx_kwargs['worker_pid'] = worker_pid
    if result.current_worker:
        ctx_kwargs['worker_pid'] = result.current_worker.pid
        if not plugin_id and result.current_worker.plugin_id:
            ctx_kwargs['plugin_id'] = result.current_worker.plugin_id
    
    ctx_kwargs['operation'] = 'validate_resources'
    
    # Add extra context
    if extra_context:
        ctx_kwargs['extra'] = extra_context
    
    # Add conflict info to extra context
    if result.conflict:
        if 'extra' not in ctx_kwargs:
            ctx_kwargs['extra'] = {}
        ctx_kwargs['extra']['resource_status'] = result.conflict.status.value
        ctx_kwargs['extra']['resource_type'] = result.conflict.resource_type.value
        if result.conflict.app_pids:
            ctx_kwargs['extra']['app_pids'] = result.conflict.app_pids
        if result.conflict.external_pids:
            ctx_kwargs['extra']['external_pids'] = result.conflict.external_pids
    
    context = ErrorContext(**ctx_kwargs)
    
    # Success cases - no error needed
    if result.action in (ValidationAction.PROCEED, ValidationAction.RELOAD_PLUGIN):
        return None
    
    # Determine error type and properties based on action
    if result.action == ValidationAction.ABORT:
        return ValidationError(
            message=result.message,
            context=context,
            severity=ErrorSeverity.ERROR,
            is_retryable=False
        )
    
    elif result.action == ValidationAction.USER_INTERVENTION:
        # External process conflict - needs user action
        suggested_action = "Stop external GPU processes to proceed"
        return ResourceError(
            message=result.message,
            context=context,
            severity=ErrorSeverity.WARNING,
            is_retryable=False,  # Won't fix itself
            resource_type="GPU",
            suggested_action=suggested_action
        )
    
    elif result.action in (ValidationAction.WAIT_FOR_JOB, ValidationAction.WAIT_FOR_OTHER_WORKER):
        # Resource temporarily busy - retryable
        if result.action == ValidationAction.WAIT_FOR_JOB:
            suggested_action = "Wait for current job to complete or cancel it"
        else:
            suggested_action = f"Wait for {result.current_worker.worker_type} worker to finish or stop it"
        
        return ResourceError(
            message=result.message,
            context=context,
            severity=ErrorSeverity.WARNING,
            is_retryable=True,  # Transient conflict
            resource_type="GPU",
            suggested_action=suggested_action
        )
    
    # Shouldn't reach here, but handle gracefully
    return ValidationError(
        message=result.message,
        context=context,
        severity=ErrorSeverity.ERROR,
        is_retryable=False
    )

### Example: Converting ValidationResult to Errors

This shows how to use the error handling integration:

In [ ]:
# Example 1: ABORT case (plugin not found)
if _has_error_handling:
    abort_result = ValidationResult(
        action=ValidationAction.ABORT,
        can_proceed=False,
        message="Plugin transcription_whisper_huge not found."
    )
    
    error = validation_result_to_error(
        abort_result,
        plugin_id="transcription_whisper_huge",
        job_id="job-123"
    )
    
    print("Example 1: ABORT -> ValidationError")
    print(f"  Error type: {type(error).__name__}")
    print(f"  Message: {error.get_user_message()}")
    print(f"  Retryable: {error.is_retryable}")
    print(f"  Severity: {error.severity.value}")
    print(f"  Context: plugin_id={error.context.plugin_id}, job_id={error.context.job_id}")
else:
    print("cjm-error-handling not installed - skipping example")

Example 1: ABORT -> ValidationError
  Error type: ValidationError
  Message: Plugin transcription_whisper_huge not found.
  Retryable: False
  Severity: error
  Context: plugin_id=transcription_whisper_huge, job_id=job-123


In [ ]:
# Example 2: WAIT_FOR_JOB case (GPU busy with same worker type)
if _has_error_handling:
    # Simulate a worker state
    from cjm_fasthtml_resources.core.manager import WorkerState
    
    busy_worker = WorkerState(
        pid=54321,
        worker_type="transcription",
        job_id="job-current",
        plugin_id="whisper_large",
        plugin_name="whisper_large",
        loaded_plugin_resource="openai/whisper-large-v3",
        config={"model_id": "openai/whisper-large-v3"},
        status="running"
    )
    
    wait_result = ValidationResult(
        action=ValidationAction.WAIT_FOR_JOB,
        can_proceed=False,
        message="GPU in use by running job (PID 54321). Wait for completion or cancel job.",
        current_worker=busy_worker
    )
    
    error = validation_result_to_error(
        wait_result,
        plugin_id="whisper_base",
        job_id="job-456"
    )
    
    print("\nExample 2: WAIT_FOR_JOB -> ResourceError")
    print(f"  Error type: {type(error).__name__}")
    print(f"  Message: {error.get_user_message()}")
    print(f"  Retryable: {error.is_retryable}")
    print(f"  Resource type: {error.resource_type}")
    print(f"  Suggested action: {error.suggested_action}")
    print(f"  Context worker PID: {error.context.worker_pid}")
else:
    print("cjm-error-handling not installed - skipping example")


Example 2: WAIT_FOR_JOB -> ResourceError
  Error type: ResourceError
  Message: GPU in use by running job (PID 54321). Wait for completion or cancel job.
  Retryable: True
  Resource type: GPU
  Suggested action: Wait for current job to complete or cancel it
  Context worker PID: 54321


In [ ]:
# Example 3: Practical usage pattern - raise error if validation fails
if _has_error_handling:
    print("\nExample 3: Practical Usage Pattern")
    print("="*60)
    
    def start_job_with_validation(plugin_id, job_id, validation_result):
        """Example function showing how to use validation + errors together."""
        # Check if we can proceed
        if not validation_result.can_proceed:
            # Convert to error and raise
            error = validation_result_to_error(
                validation_result,
                plugin_id=plugin_id,
                job_id=job_id
            )
            raise error
        
        # Validation passed, proceed with job
        return f"Job {job_id} started successfully with {plugin_id}"
    
    # Test with ABORT case
    try:
        result = ValidationResult(
            action=ValidationAction.ABORT,
            can_proceed=False,
            message="Plugin not found"
        )
        start_job_with_validation("whisper_huge", "job-789", result)
    except ValidationError as e:
        print(f"Caught ValidationError: {e.get_user_message()}")
        print(f"  Action: Don't retry, fix the plugin ID")
    
    # Test with WAIT_FOR_JOB case (retryable)
    try:
        result = ValidationResult(
            action=ValidationAction.WAIT_FOR_JOB,
            can_proceed=False,
            message="GPU busy"
        )
        start_job_with_validation("whisper_base", "job-999", result)
    except ResourceError as e:
        print(f"\nCaught ResourceError: {e.get_user_message()}")
        print(f"  Retryable: {e.is_retryable}")
        print(f"  Action: Retry after GPU becomes available")
    
    print("\n" + "="*60)
else:
    print("cjm-error-handling not installed - skipping example")


Example 3: Practical Usage Pattern
Caught ValidationError: Plugin not found
  Action: Don't retry, fix the plugin ID

Caught ResourceError: GPU busy
  Retryable: True
  Action: Retry after GPU becomes available



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()